<a href="https://colab.research.google.com/github/Nuwantha97/Sinhala_spell_and_grammer_checker/blob/Notebooks/Spell_checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



Spelling Errors:
Focus on the correct arrangement of letters in a word to match its standard or dictionary form.
- Example: "adres" instead of "address."

Grammar Errors:
Focus on the syntax, word forms, and sentence structure to convey proper meaning and adhere to language rules.
- Example: "He going to school yesterday" instead of "He went to school yesterday."

#Edit Distance (Levenshtein Distance)


The Levenshtein library in Python is a specialized tool for computing Levenshtein distances

- Levenshtein Distance: Calculates the minimum edit distance between two strings.
- Levenshtein Similarity: Measures how similar two strings are, typically on a scale from 0 to 1.
- Other Metrics:
 - Ratio: A normalized version of the distance (1 - distance/max length).
 - Hamming Distance: Number of positions where two strings of equal length differ.
 - Jaro-Winkler Similarity: A more nuanced similarity metric, especially for short strings.

In [2]:
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.3 MB/s eta 0:00:00


# Create Sinhala dictionary (dataset 01)

## Split word from sentence

In [9]:
# Open the input text file in read mode
with open('/content/drive/MyDrive/Projects/Sinhala Spell and Grammer checker/Datasets/sinhala_full_word_list_2016-10-08.txt', 'r', encoding='utf-8') as infile:
    # Read all lines from the input file
    lines = infile.readlines()

# Create a list to store all words
words = []

# Loop through each line to extract words
for line in lines:
    # Split the line into words and extend the list
    words.extend(line.split())


## Remove dublicate words

In [10]:
unique_words = []
seen = set()
for word in words:
    if word not in seen:
        unique_words.append(word)
        seen.add(word)

## Remove Non sinhala words

In [11]:
import re

# Sinhala Unicode character range
sinhala_pattern = re.compile(r'^[\u0D80-\u0DFF]+$')

# Filter Sinhala words
sinhala_words = [word for word in unique_words if sinhala_pattern.match(word)]

In [12]:
import csv

# Open the output CSV file in write mode
with open('/content/drive/MyDrive/Projects/Sinhala Spell and Grammer checker/Datasets/sinhala_dict1.csv', 'w', encoding='utf-8', newline='') as outfile:
    writer = csv.writer(outfile)

    # Write each word as a new row in the CSV
    for word in sinhala_words:
        writer.writerow([word])

print("Words have been written to 'sinhala_dict1.csv' line by line.")


Words have been written to 'sinhala_dict1.csv' line by line.


# Dataset 02

In [13]:
import pandas as pd

# Load the .xlsx file
file_path = "/content/drive/MyDrive/Projects/Sinhala Spell and Grammer checker/Datasets/data-spell-checker.xlsx"  # Replace with your file's path
data = pd.read_excel(file_path)

# Filter the rows where label == 1
filtered_data = data[data['label'] == 1]

# Select only the words column
words_with_label_1 = filtered_data['word']

# Save the filtered words to a .csv file
output_file_path = "/content/drive/MyDrive/Projects/Sinhala Spell and Grammer checker/Datasets/sinhala_dict2.csv"  # Replace with your desired output file name
words_with_label_1.to_csv(output_file_path, index=False, header=False)

print(f"Filtered words saved to {output_file_path}")


Filtered words saved to /content/drive/MyDrive/Projects/Sinhala Spell and Grammer checker/Datasets/sinhala_dict2.csv


# Spell check

In [6]:
# Function to load the dictionary from a text file
def load_dictionary(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        dictionary = [line.strip() for line in file]
    return dictionary

In [7]:
import Levenshtein

def spell_check(word, dictionary, top_n=3):
    # List to store words with their distances
    word_distances = []

    for correct_word in dictionary:
        # Calculate the Levenshtein distance between the word and dictionary word
        distance = Levenshtein.distance(word, correct_word)
        word_distances.append((correct_word, distance))

    # Sort the list by distance (ascending order)
    word_distances.sort(key=lambda x: x[1])

    # Return the top N closest words
    return word_distances[:top_n]

def check_sentence(sentence, sinhala_dictionary):
    words = sentence.split()  # Split the input sentence into words
    corrected_words = []  # List to store corrected words
    distances = []  # List to store Levenshtein distances for each word

    for word in words:
        # Get the top suggestion (closest word) and its distance
        top_words = spell_check(word, sinhala_dictionary, top_n=3)
        if top_words:  # Ensure there's at least one suggestion
            corrected_word, distance = top_words[0]  # Top suggestion
            corrected_words.append(corrected_word)  # Add corrected word
            distances.append(distance)  # Add the distance
        else:
            # If no suggestions, append the original word
            corrected_words.append(word)
            distances.append(None)  # No distance available

        print_suggestion(word, top_words)

    # Combine corrected words into a single sentence
    corrected_sentence = ' '.join(corrected_words)

    # Return values
    return sentence, corrected_sentence, distances

def print_suggestion(word, top_words):
    print(f"Suggestions for '{word}':")
    for i, (correct_word, distance) in enumerate(top_words, 1):
        print(f"{i}. {correct_word} (Distance: {distance})")

In [23]:
# Load the dictionary from the text file
sinhala_dictionary = load_dictionary('/content/drive/MyDrive/Projects/Sinhala Spell and Grammer checker/Datasets/sinhala_dict2.csv')

# Input word
sentence = "මම පොත් ලයනවා"

# Perform spell check
sentence, corrected_sentence, distances = check_sentence(sentence, sinhala_dictionary)
print(f"\nInput Sentence: {sentence}")
print(f"Suggested Correction: {corrected_sentence}")
print(f"Levenshtein Distances: {distances}")

Suggestions for 'මම':
1. මම (Distance: 0)
2. ඇම (Distance: 1)
3. ඉම (Distance: 1)
Suggestions for 'පොත්':
1. පහත් (Distance: 1)
2. පොතන (Distance: 1)
3. පොත්ත (Distance: 1)
Suggestions for 'ලයනවා':
1. උයනවා (Distance: 1)
2. ගයනවා (Distance: 1)
3. යනවා (Distance: 1)

Input Sentence: මම පොත් ලයනවා
Suggested Correction: මම පහත් උයනවා
Levenshtein Distances: [0, 1, 1]
